In [ ]:
import sys
from utlis_func import *

%matplotlib inline
seaborn.set(font_scale=2)
seaborn.set_style("white")

In [ ]:
# random points at least 2r apart
n = 15
num_test = 10000
# define margin
margin = 0.25

In [ ]:
X_test, y_test = generate_sample(num_test, margin)
visualize_data(X_test, y_test, margin)

## Plotting for different ep

In [ ]:
num_trial = 30
eps = np.linspace(0, margin*2, 10*2)
b_ls = np.zeros((num_trial, len(eps)))
v_ls = np.zeros((num_trial, len(eps)))
r_ls = np.zeros((num_trial, len(eps)))
nets0 = {}
nets1 = {}
for j in range(num_trial):
    X_0, y_0 = generate_sample(n, margin)
    X_1, y_1 = generate_sample(n, margin)

    for i in range(len(eps)):
        epsilon = eps[i]
        robust_net_0 = robust_train(X_0, y_0, epsilon, seed=1)
        robust_net_1 = robust_train(X_1, y_1, epsilon, seed=2)
        nets0[(j, i)] = robust_net_0
        nets1[(j, i)] = robust_net_1
        b, v, r = BV_compute(robust_net_0, robust_net_1, X_test, y_test)
        b_ls[j,i] = b
        v_ls[j,i] = v
        r_ls[j,i] = r
        print('finished eps={}'.format(eps[i]))
        print('============================================================')

In [ ]:
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

vp_ls = v_ls[:,:10]
bp_ls = b_ls[:,:10]
rp_ls = r_ls[:,:10]
eps = eps[:10]
cv = 0.6*np.std(vp_ls, 0)
plt.fill_between(eps, np.mean(vp_ls, 0)-cv, np.mean(vp_ls, 0)+cv, color='orangered', alpha=.1)
plt.plot(eps, np.mean(vp_ls, 0), '-d', color='orangered', label = "Variance", linewidth=2.5, markeredgecolor='k', markersize=6)

cb = 0.6*np.std(bp_ls, 0)
plt.fill_between(eps, np.mean(bp_ls, 0)-cb, np.mean(bp_ls, 0)+cb, color='royalblue', alpha=.1)
plt.plot(eps, np.mean(bp_ls, 0), '-o', color='royalblue', label = "Bias", linewidth=2.5, markeredgecolor='k', markersize=6)

cr = 0.6*np.std(rp_ls, 0)
plt.fill_between(eps, np.mean(rp_ls, 0)-cr, np.mean(rp_ls, 0)+cr, color='forestgreen', alpha=.1)
plt.plot(eps, np.mean(rp_ls, 0), '-s', color='forestgreen', label = "Risk", linewidth=2.5, markeredgecolor='k', markersize=6)

plt.axhline(y=0.0, color='black', linestyle='--')
plt.axhline(y=np.mean(vp_ls, 0)[0], color='black', linestyle=':')

plt.ylabel('Bias/Variance/Risk', fontsize=17)
plt.xlabel('Adversarial Training EPS (PGD)', fontsize=17)
plt.legend(loc='best', prop={"size": 15}, framealpha=0.5)
plt.tight_layout()
plt.show()